# Pathology prediction (Enlarged_CM)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the Enlarged Cardiomediastinum pathology using the CSV embeddings file



## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Enlarged Cardiomediastinum pathology is : (N =  3206)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.ENLARGED_CARDIOMEDIASTINUM,
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="EnlargedCardiomediastinum", experiment_name="CP_EnlargedCardiomediastinum", filepath="./results/enlargedcardiomediastinum")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-26 11:11:21,998	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=409777) Outer fold 2
(run_fold pid=409777) Configuring PyCaret for outer fold 2
(run_fold pid=409766) Outer fold 4 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [02:45<01:01, 61.29s/it] [repeated 4x across cluster]


(run_fold pid=409766)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=409766) Fold                                                          
(run_fold pid=409766) 0       0.8370  0.8827  0.9379  0.8567  0.8955  0.5287  0.5401
(run_fold pid=409766) 1       0.8394  0.8575  0.9608  0.8448  0.8991  0.5140  0.5405
(run_fold pid=409766) 2       0.8537  0.8810  0.9443  0.8701  0.9057  0.5820  0.5918
(run_fold pid=409766) 3       0.8415  0.8587  0.9216  0.8731  0.8967  0.5574  0.5612
(run_fold pid=409766) 4       0.8366  0.8807  0.9379  0.8567  0.8955  0.5247  0.5361
(run_fold pid=409766) Mean    0.8416  0.8721  0.9405  0.8603  0.8985  0.5414  0.5539
(run_fold pid=409766) Std     0.0063  0.0115  0.0126  0.0102  0.0038  0.0249  0.0208
(run_fold pid=409778) Configuring PyCaret for outer fold 1 [repeated 3x across cluster]
(run_fold pid=409766) Tuning hyperparameters for model xgboost with custom grid using grid search


                                                         
Processing:  75%|███████▌  | 3/4 [02:46<01:01, 61.57s/it] [repeated 3x across cluster]


(run_fold pid=409770) 0       0.8491  0.8723  0.9608  0.8547  0.9046  0.5500  0.5722
(run_fold pid=409770) 1       0.8394  0.8769  0.9444  0.8550  0.8975  0.5309  0.5452
(run_fold pid=409770) 2       0.8341  0.8788  0.9410  0.8516  0.8941  0.5164  0.5303
(run_fold pid=409770) 3       0.8366  0.8694  0.9443  0.8521  0.8958  0.5218  0.5369
(run_fold pid=409770) 4       0.8341  0.8473  0.9346  0.8563  0.8938  0.5192  0.5297
(run_fold pid=409777) 0       0.8297  0.8504  0.9346  0.8512  0.8910  0.5059  0.5177
(run_fold pid=409777) 1       0.8662  0.9081  0.9706  0.8659  0.9153  0.6022  0.6250
(run_fold pid=409777) 2       0.8390  0.8627  0.9377  0.8589  0.8966  0.5370  0.5477
(run_fold pid=409777) 3       0.8415  0.8996  0.9377  0.8614  0.8980  0.5456  0.5556
(run_fold pid=409777) 4       0.8463  0.8605  0.9837  0.8384  0.9053  0.5121  0.5616
(run_fold pid=409778) 0       0.8637  0.8928  0.9608  0.8698  0.9130  0.6020  0.6182
(run_fold pid=409778) 1       0.8293  0.8623  0.9279  0.8550  0.8

Processing:   0%|          | 0/4 [00:00<?, ?it/s]
                                                          [repeated 3x across cluster]
Processing:  25%|██▌       | 1/4 [00:00<00:02,  1.40it/s]


(run_fold pid=409777) Transformation Pipeline and Model Successfully Saved
(run_fold pid=409777)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=409777) 0  Extreme Gradient Boosting    0.8658  0.9142  ...  0.9143  0.6082  0.6233
(run_fold pid=409777) 
(run_fold pid=409777) [1 rows x 8 columns]
(run_fold pid=409766) Transformation Pipeline and Model Successfully Saved
(run_fold pid=409766)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=409766) 0  Extreme Gradient Boosting    0.8471  0.8904  ...  0.9016  0.5617  0.5718
(run_fold pid=409766) 
(run_fold pid=409766) [1 rows x 8 columns]


Processing:  75%|███████▌  | 3/4 [02:33<00:56, 56.59s/it]
                                                         


(run_fold pid=409778)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=409778) Fold                                                          
(run_fold pid=409778) 0       0.8540  0.8860  0.9444  0.8705  0.9060  0.5823  0.5921
(run_fold pid=409778) 1       0.8516  0.8758  0.9346  0.8746  0.9036  0.5824  0.5885
(run_fold pid=409778) 2       0.8561  0.8756  0.9377  0.8773  0.9065  0.5958  0.6021
(run_fold pid=409778) 3       0.8537  0.8907  0.9314  0.8796  0.9048  0.5901  0.5946
(run_fold pid=409778) 4       0.8390  0.8829  0.9346  0.8614  0.8966  0.5366  0.5458
(run_fold pid=409778) Mean    0.8509  0.8822  0.9366  0.8727  0.9035  0.5774  0.5846
(run_fold pid=409778) Std     0.0061  0.0058  0.0044  0.0064  0.0036  0.0210  0.0199
(run_fold pid=409778) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=409770) Transformation Pipeline and Model Successfully Saved
(run_fold pid=409770)                        Model  Accuracy  